# Convolutional Neural Network

### Importing the libraries

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
tf.__version__

'2.5.0-rc0'

## Part 1 - Data Preprocessing

Check out this tutorial on [image data augmentation](https://stepup.ai/exploring_data_augmentation_keras/).

### Preprocessing the Training set

[keras image preprocessing API](https://keras.io/api/preprocessing/image/)
 
We use the [ImageDataGenerator](https://keras.io/api/preprocessing/image/#imagedatagenerator-class) which can generate batches of data with real time transformations, as defined in the construction of the generator object.  

The rescale parameter set to this specific value (1/255) provides [feature scaling](https://en.wikipedia.org/wiki/Feature_scaling): normalization of the input.  
Since each pixel's value is in the range [0 - 255], dividing by 255 will map the values to the [0 - 1] range.

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# target_size: the size of the images when loaded
# here we resize the images to 64x64 pixels
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

When loading the test set, we don't perform any transformations, since we want to predict the class of the actual images. (That's why they're called test images. It wouldn't make much sense trying to predict the classes of different images, which is what we obtain after the augmentation process).  

It is crucial, however, to perform feature scaling, since we've done so when we processed the training images.  
Skipping this step would result in trying to predict image classes for images that are entirely on a different value scale, and the accuracy would be, as a result, very low.

In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255)

# class_mode: binary/categorical
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [5]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

`filters`: the number of feature detectors (a.k.a. kernels / filters) you want to apply to the input.  
`input_shape = [64, 64, 3]`: 64x64 pixels, 3 channels (RGB). This parameter is only needed for the **first layer**. 
                             Additional convolutional layers can infer the size from the layer connected to it.

In [6]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [7]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [9]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

`units`: the number of neurons in the layer connected directly to the output of the CNN.

In [10]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

`activation`: the activation function.
For binary classification, use sigmoid.  
For categorical classification, use softmax.

In [11]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [12]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [32]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 60s 239ms/step - loss: 0.7126 - accuracy: 0.5262 - val_loss: 0.6328 - val_accuracy: 0.6230
Epoch 2/25
250/250 [==============================] - 63s 251ms/step - loss: 0.6364 - accuracy: 0.6313 - val_loss: 0.6072 - val_accuracy: 0.6680
Epoch 3/25
250/250 [==============================] - 65s 259ms/step - loss: 0.5891 - accuracy: 0.6868 - val_loss: 0.5765 - val_accuracy: 0.6965
Epoch 4/25
250/250 [==============================] - 66s 266ms/step - loss: 0.5440 - accuracy: 0.7235 - val_loss: 0.5910 - val_accuracy: 0.7080
Epoch 5/25
250/250 [==============================] - 67s 268ms/step - loss: 0.5227 - accuracy: 0.7433 - val_loss: 0.5072 - val_accuracy: 0.7475
Epoch 6/25
250/250 [==============================] - 67s 267ms/step - loss: 0.4870 - accuracy: 0.7708 - val_loss: 0.4891 - val_accuracy: 0.7595
Epoch 7/25
250/250 [==============================] - 67s 270ms/step - loss: 0.4714 - accuracy: 0.7695 - val_loss: 0.4717 - val_ac

## Part 4 - Making a single prediction

**Note:**  
`expand_dims` is needed to add an extra dimension to the input image, since we've trained the model in batch mode.  Meaning, the first dimension of the input is the batch axis.  
In this case, we're predicting the label of a single image, so the first dimension has only one call. (whereas during training, the first dimension has 32 cell, or more generally, `batch_size` cells).

The `class_indices` method returns the index assignment for each class (sometimes referred to as categore or label).  

In [13]:
training_set.class_indices

{'cats': 0, 'dogs': 1}

In [0]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image/255.0) # normalize the image. This is necessary because training was done on normalized input.
training_set.class_indices

if result[0][0] > 0.5:
  prediction = 'cat'
else:
  prediction = 'dog'

Why the `> 0.5` ?  
The [tf.kera.Model.predict](https://www.tensorflow.org/api_docs/python/tf/keras/Model#predict) method returns an array of predictions.  
The size of the array matches the number of categories (in this example it's 2).  
Each cell contains the **prediction confidence**; how sure the model is that the input belongs to a certain category.  

For example, a prediction might look like this: `[0.95, 0.05]`, meaning the model is 95% sure that the image is of a cat (index 0 corresponds to the label 'cat' according to `class_indices`.

In [0]:
print(prediction)